# `System.Text.Json` with F#

The most unpopular way to deserialize types in F# is to explicitly express the intent to traverse the document looking for values. My other studies of JSON in F# in [this directory](../json) show the more popular ways to get this done.

I am so devoted to expressing explicit traversal that I built [a module of functions](https://github.com/BryanWilhite/Songhay.Modules/blob/853c780e1f4aa278426d924b429f0f45c7950f9a/Songhay.Modules/JsonDocumentUtility.fs) dedicated to this miserable task:

In [1]:
#!fsharp

open System.Linq
open System.Text.Json

/// <summary>
/// Wraps <see cref="JsonException" /> property
/// in <see cref="Error" />.
/// </summary>
/// <param name="elementName">The <see cref="JsonElement" /> name.</param>
let resultError (elementName: string) =
    Error(JsonException $"the expected `{elementName}` element is not here.")

/// <summary>
/// Tries to return the <see cref="JsonElement" /> property
/// of the specified <see cref="JsonElement" /> object.
/// </summary>
/// <param name="elementName">The <see cref="JsonElement" /> name.</param>
/// <param name="element">The <see cref="JsonElement" />.</param>
let tryGetProperty (elementName: string) (element: JsonElement) =
    match element.TryGetProperty elementName with
    | false, _ -> resultError elementName
    | true, el -> Ok el

/// <summary>
/// Converts the <see cref="JsonElement" />
/// to its property name or returns <see cref="None" />.
/// </summary>
/// <param name="element">The <see cref="JsonElement" />.</param>
let toPropertyName (element: JsonElement) =
    if element.ValueKind <> JsonValueKind.Object then None
    else
        try
            Some (element.EnumerateObject().First().Name)
        with | _ -> None


## the importance of `tryGetProperty`

The `tryGetProperty` function shown above is a functional wrapper around `JsonElement.TryGetProperty` [ 📖[docs](https://docs.microsoft.com/en-us/dotnet/api/system.text.json.jsonelement.trygetproperty?view=net-7.0)], formally expressing that trying to get a “child” child element will always return `Result<JsonElement,JsonException>` where the exception is centralized by `resultError`.

For additional safety and rich communication, we can verify that we are traversing the expected document by using the `toPropertyName` function before we even try `tryGetProperty`.

In [2]:
#!fsharp

let jDoc = JsonDocument.Parse(@"
    {
        ""top"": {
            ""one"": ""this is first"",
            ""two"": ""this is second"",
            ""three"": {
                ""p1"": ""this is three-point-one"",
                ""p2"": ""this is three-point-two""
            }
        }
    }
")

For example, we can verify that the chunk of JSON above has a root element called `top`:

In [3]:
#!fsharp

jDoc.RootElement |> toPropertyName

Value,top


The result above leads me to assume that `RootElement` _is_ the element `top` and this would make me express the following and not see an error:

In [4]:
#!fsharp

jDoc.RootElement |> tryGetProperty "one"

Microsoft.FSharp.Core.FSharpResult`2[System.Text.Json.JsonElement,System.Text.Json.JsonException] ResultValue ValueKind Undefined ErrorValue System.Text.Json.JsonException: the expected `one` element is not here. LineNumber <null> BytePositionInLine <null> Path <null> Message the expected `one` element is not here. TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233088 StackTrace <null>

But the result _is_ an error! However, the following gets the expected result:

In [5]:
#!fsharp

jDoc.RootElement |> tryGetProperty "top" |> Result.bind (tryGetProperty "one")

ResultValue,"""this is first"""
ErrorValue,<null>


What is going on here? Before I try to answer the question, let me take a look at the properties of `top.three`:

In [6]:
#!fsharp

open System.Linq

let result =
    jDoc.RootElement
    |> tryGetProperty "top"
    |> Result.bind (tryGetProperty "three")
    |> Result.map (fun element -> element.EnumerateObject().ToArray())

result |> Result.defaultValue [||]

index value 0 "p1": "this is three-point-one" Value "this is three-point-one" Name p1 1 "p2": "this is three-point-two" Value "this is three-point-two" Name p2

Notice that `EnumerateObject` [📖 [docs](https://docs.microsoft.com/en-us/dotnet/api/system.text.json.jsonelement.enumerateobject?view=net-7.0)] for `top.three` is listing its children while `RootElement.EnumerateObject` is listing itself:

In [7]:
#!fsharp

jDoc.RootElement.EnumerateObject().ToArray()

index value 0 "top": {\n "one": "this is first",\n "two": "this is second",\n "three": {\n "p1": "this is three-point-one",\n "p2": "this is three-point-two"\n }\n } Value {
 "one": "this is first",
 "two": "this is second",
 "three": {
 "p1": "this is three-point-one",
 "p2": "this is three-point-two"
 }
 } one "this is first" two "this is second" three {
 "p1": "this is three-point-one",
 "p2": "this is three-point-two"
 } p1 "this is three-point-one" p2 "this is three-point-two" Name top


My `tryGetProperty` function is not taking account of this difference. Now that I see how `EnumerateObject` works _differently_, I can finally see that my `toPropertyName` only works correctly for `top` as it returns `p1` for `top.three`:

In [8]:
#!fsharp

jDoc.RootElement
    |> tryGetProperty "top"
    |> Result.bind (tryGetProperty "three")
    |> Result.map (fun element -> element |> toPropertyName)
    |> Result.defaultValue (Some "nope.")

Value,p1


When I started writing this notebook, I did not expect to find such devastating bugs 🐛 But here we are.

## addressing my bugs 🐛

In the time-frame before .NET 6, there is no way to determine whether an instance of `JsonElement` is `RootElement` without having a reference to `JsonDocument` hanging around. A [GitHub issue for Microsoft](https://github.com/dotnet/runtime/issues/40452) explicitly proposed that there should be some way to the find the parent of `JsonElement`. This proposal was rejected in favor of celebrating the `System.Text.Json.Nodes.JsonNode` [📖 [docs](https://docs.microsoft.com/en-us/dotnet/api/system.text.json.nodes.jsonnode?view=net-6.0)] released with .NET 6.0.

I am going to address my embarrassing bugs by responding to these assertions:

1. `toPropertyName` cannot work as intended with `JsonElement`.
2. `tryGetProperty` needs to behave differently for a `JsonElement` that is a child of `JsonDocument`.

My response to _1_ is to redefine `toPropertyName`, expressing the intent that this function only applies to `JsonDocument`:

In [9]:
#!fsharp

let toPropertyName (document: JsonDocument) =
    if document.RootElement.ValueKind <> JsonValueKind.Object then None
    else
        try
            Some (document.RootElement.EnumerateObject().First().Name)
        with | _ -> None


For _2_, I define a new type, `JsonDocumentOrElement`, to clearly express that this function takes `JsonDocument` or `JsonElement`:

In [10]:
#!fsharp

type JsonDocumentOrElement =
    | JDocument of JsonDocument
    | JElement of JsonElement

let toJsonElement (documentOrElement: JsonDocumentOrElement) =
        match documentOrElement with
        | JDocument doc -> doc.RootElement
        | JElement el -> el

let tryGetProperty (elementName: string) (documentOrElement: JsonDocumentOrElement) =

    let tryGet (name: string) (element: JsonElement) =
        match element.TryGetProperty name with
        | false, _ -> resultError name
        | true, el -> Ok (JElement el)

    match documentOrElement with
    | JElement element -> element |> tryGet elementName
    | JDocument document ->
        match document |> toPropertyName with
        | None _ -> resultError elementName
        | Some rootName ->
            match document.RootElement.TryGetProperty rootName with
            | false, _ -> resultError elementName
            | true, el -> el |> tryGet elementName


This new version of `tryGetProperty` actually contains the old version of the function—the badly-named `tryGet` function. This new version of `tryGetProperty` actually produces the expected result:

In [11]:
#!fsharp

let elementOne = JDocument jDoc |> tryGetProperty "one" |> Result.map (toJsonElement)

elementOne

ResultValue,"""this is first"""
ErrorValue,<null>


I _feel_ that I can refine my new, bulky version of `tryGetProperty` by taking advantage of the recursion feature (`rec`) in F#:

In [12]:
#!fsharp

let rec tryGetProperty (elementName: string) (documentOrElement: JsonDocumentOrElement) =
    match documentOrElement with
    | JElement element ->
        match element.TryGetProperty elementName with
        | false, _ -> resultError elementName
        | true, el -> Ok (JElement el)
    | JDocument document ->
        match document |> toPropertyName with
        | None _ -> resultError elementName
        | Some rootName ->
            match document.RootElement.TryGetProperty rootName with
            | false, _ -> resultError elementName
            | true, el -> JElement el |> tryGetProperty elementName

In [13]:
#!fsharp

let elementOne = JDocument jDoc |> tryGetProperty "one" |> Result.map (toJsonElement)

elementOne

ResultValue,"""this is first"""
ErrorValue,<null>


Now I can bask in triumphant misery, traversing the JSON document with expectations met:

In [14]:
#!fsharp

let elementP1 =
    JDocument jDoc
    |> tryGetProperty "three"
    |> Result.bind (tryGetProperty "p1")
    |> Result.map (toJsonElement)

elementP1

ResultValue,"""this is three-point-one"""
ErrorValue,<null>


@[BryanWilhite](https://twitter.com/BryanWilhite)
